# Simulation analysis

In [16]:
import pandas as pd
import pickle
from os import listdir
from os.path import join
import plotly.graph_objects as go
import regex as re
import itertools
import numpy as np
import plotly.express as px

COLORS = px.colors.qualitative.Plotly
IMAGE_SCALE = 3
STEPS_IN_HOUR = 120
AGGREGATION_PARAMETER = 60
INPUT_PATH = 'pickle'

In [17]:
data_collector_files = [f for f in listdir(INPUT_PATH) if re.match('datacollector', f)]
data_collector_files

['datacollector$coda1_jockey1$1631545130.pkl',
 'datacollector$coda1_jockey1_5$1631545126.pkl',
 'datacollector$coda1_jockey1_6$1631545130.pkl',
 'datacollector$coda1_jockey1_7$1631545118.pkl',
 'datacollector$coda1_jockey1_8$1631545128.pkl',
 'datacollector$coda1_jockey2$1631545355.pkl',
 'datacollector$coda2_jockey1$1631545117.pkl',
 'datacollector$coda2_jockey2$1631545074.pkl',
 'datacollector$coda3_jockey1$1631545120.pkl',
 'datacollector$coda3_jockey2$1631545112.pkl',
 'datacollector$coda4_jockey1$1631545126.pkl',
 'datacollector$coda4_jockey2$1631545091.pkl',
 'datacollector$codacondivisa_10$1631545095.pkl',
 'datacollector$codacondivisa_5$1631545104.pkl',
 'datacollector$codacondivisa_6$1631545069.pkl',
 'datacollector$codacondivisa_7$1631545060.pkl',
 'datacollector$codacondivisa_8$1631545065.pkl',
 'datacollector$prob_coda1_jockey1$1631545121.pkl',
 'datacollector$prob_coda1_jockey2$1631545115.pkl',
 'datacollector$prob_coda2_jockey1$1631545130.pkl',
 'datacollector$prob_coda2

In [18]:
def read_pickle_file(filename):
    with open(join(INPUT_PATH, filename), 'rb') as f:
        return pickle.load(f)

def read_simulation(filename, df_arrivals, steps_in_hour=STEPS_IN_HOUR):
    simulation_dict = read_pickle_file(filename)
    simulation_df = pd.DataFrame(simulation_dict)
    # Add simulation name
    simulation_name = filename.split('$')[1]
    full_length = steps_in_hour * len(df_arrivals)
    q, r = len(simulation_df) // full_length, len(simulation_df) % full_length
    simulation_df["simulation_name"] = [simulation_name] * (q * full_length + r)
    # Add hours
    hours = [([hour]*steps_in_hour) for hour in df_arrivals["hour"]] * q
    hours = hours + [[df_arrivals["hour"].iloc[-1]] * (len(simulation_df) % full_length)]
    simulation_df["hour"] = list(itertools.chain(*hours))

    return simulation_df

df_arrivals = read_pickle_file('df_arrivals_aggregated.pkl')

df_simulations = pd.concat([read_simulation(x, df_arrivals) for x in data_collector_files])
df_simulations.head()

,Total_customers,Density_total,Flow_total,Density_standard,Flow_standard,Density_self_scan,Flow_self_scan,Total_steps,Avg_waiting_times_standard,Avg_waiting_times_self_scan,Number_exiting_customers,simulation_name,hour
0,1,0.142857,0.142857,0.142857,0.142857,0.0,0.0,2,0.0,0.0,0,coda1_jockey1,8
1,2,0.285714,0.142857,0.285714,0.142857,0.0,0.0,3,0.0,0.0,0,coda1_jockey1,8
2,3,0.428571,0.142857,0.428571,0.142857,0.0,0.0,4,0.0,0.0,0,coda1_jockey1,8
3,4,0.571429,0.142857,0.571429,0.142857,0.0,0.0,5,0.0,0.0,0,coda1_jockey1,8
4,5,0.714286,0.142857,0.714286,0.142857,0.0,0.0,6,0.0,0.0,0,coda1_jockey1,8


In [19]:
aggregate_hours = lambda df: df.groupby(by=['hour']).mean().reset_index()

def aggregate_steps(df, factor):
    df['aux'] = df.reset_index()['index'].apply(lambda v: int(np.floor(v / factor) * factor))
    return aggregate_hours(df).drop(columns=['aux'])


def add_multiple_traces(fig, df_simulations, names, x_axis, y_axis, aggregation_parameter):
    for n in names:
        target_simulation = df_simulations.query(f'simulation_name == "{n}"')
        target_simulation = aggregate_steps(target_simulation, aggregation_parameter)
        x1, y1 = (target_simulation[x_axis], target_simulation[y_axis])

        if re.match('Avg_waiting', feature):
            y_values = df_simulations[feature] * (60 * 60 / STEPS_IN_HOUR)
        else:
            y_values = df_simulations[feature]

        fig.add_trace(go.Scatter(x=x1, y=y1, mode='markers', name=make_pretty_name(n)))
    return fig

def add_simulation_to_plot(
    fig, df_simulations, simulation_name, feature,
    x_axis="hour", normalize=False, line=None, mode='lines+markers'):
    if not line:
        line=dict()

    target_simulation = df_simulations.query(f'simulation_name == "{simulation_name}"')
    df = aggregate_hours(target_simulation)

    y_values = df[feature]
    if re.match('Avg_waiting', feature):
        y_values = df[feature] * (60 * 60 / STEPS_IN_HOUR / 60)
    x_values = df[x_axis]

    if normalize:
        y_values = y_values / sum(y_values)
    fig.add_trace(go.Scatter(x=x_values, y=y_values,
                             mode=mode, name=make_pretty_name(simulation_name), line=line))
    return fig

def make_pretty_name(simulation_name):
    if simulation_name[0:-2] == 'validazione':
        return 'queue strategy ' + simulation_name[-1] + ', no jockey'
    elif simulation_name[0:4] == 'coda' and simulation_name[6:12] == 'jockey':
        pretty_name = 'queue strategy ' + simulation_name[4] + ', jockey ' + simulation_name[12]
        if simulation_name[-2] == '_':
            return simulation_name[-1] + ' POS, ' + pretty_name
        return pretty_name
    elif simulation_name[0:13] == 'codacondivisa':
        pretty_name = 'N-fork'
        if simulation_name[-2] == '_':
            return simulation_name[-1] + ' POS, ' + pretty_name
        return pretty_name
    elif simulation_name == 'self_scan':
        return 'self-scan'
    elif simulation_name[0:4] == 'prob':
        return 'non det., queue strategy ' + simulation_name[9] + ', jockey ' + simulation_name[17]

def add_ground_truth(fig, df_arrivals, normalize=True):

    y_values = df_arrivals["value"]
    if normalize:
        y_values = y_values / sum(y_values)
    fig.add_trace(go.Scatter(x=df_arrivals["hour"], y=y_values,
                             mode='lines+markers', name='Ground truth'))

    return fig

def decorate_figure(fig, title='', x_axis_title='Hour',
                    y_axis_title='', normalize=False, x_dtick=1, y_dtick=1):
    if normalize:
        y_dtick = 0.01

    fig = fig.update_layout(
        xaxis = dict(
            tickmode = 'linear',
            tick0 = 0,
            dtick = x_dtick
        ),
        yaxis = dict(
            tickmode = 'linear',
            tick0 = 0,
            dtick = y_dtick
        ),
        title={
            'text': title + (' (Normalized)' if normalize else ''),
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        xaxis_title=x_axis_title,
        yaxis_title=y_axis_title + (' (Normalized)' if normalize else ''),
        legend_title="Simulation",
    )
    return fig

## Validazione


In [20]:
# Validazione

normalize = True

fig =  go.Figure()
fig = add_ground_truth(fig, df_arrivals, normalize=normalize)
fig = add_simulation_to_plot(fig, df_simulations, 'validazione_1', "Total_customers", normalize=normalize)
fig = add_simulation_to_plot(fig, df_simulations, 'validazione_2', "Total_customers", normalize=normalize)
fig = add_simulation_to_plot(fig, df_simulations, 'validazione_3', "Total_customers", normalize=normalize)
fig = add_simulation_to_plot(fig, df_simulations, 'validazione_4', "Total_customers", normalize=normalize)
fig = decorate_figure(fig,
    title="Total customers - Ground truth vs simulations for validation",
    y_axis_title="Number of incoming customers", x_dtick=1,
    normalize=normalize)
fig.show()
fig.write_image("doc/report/images/results/total_customers_validation.png", scale=IMAGE_SCALE)

## Average waiting time

In [21]:
# Average waiting time

fig =  go.Figure()

fig = add_simulation_to_plot(fig, df_simulations, 'validazione_1', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'validazione_2', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'coda1_jockey1', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'coda1_jockey2', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'coda2_jockey1', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'coda2_jockey2', "Avg_waiting_times_standard")
fig = decorate_figure(fig,
    title="Average waiting time - simulations with jockey A",
    y_axis_title="Avg waiting time (min)", x_dtick=1, y_dtick=1)

fig.show()
fig.write_image("doc/report/images/results/avg_wt_jockey_a.png", scale=IMAGE_SCALE)

## Average waiting time - No jockey

In [22]:
# Average waiting time

fig = go.Figure()

fig = add_simulation_to_plot(fig, df_simulations, 'validazione_1', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'validazione_2', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'validazione_3', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'validazione_4', "Avg_waiting_times_standard")

fig = decorate_figure(fig,
    title="Average waiting time - simulations for validation",
    y_axis_title="Avg waiting time (min)")

fig.show()
fig.write_image("doc/report/images/results/avg_wt_validation.png", scale=IMAGE_SCALE)

In [23]:
# Average waiting time

fig =  go.Figure()

fig = add_simulation_to_plot(fig, df_simulations, 'validazione_3', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'validazione_4', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'coda3_jockey1', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'coda3_jockey2', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'coda4_jockey1', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'coda4_jockey2', "Avg_waiting_times_standard")

fig = decorate_figure(fig,
    title="Average waiting time - simulations with jockey B",
    y_axis_title="Avg waiting time (min)")

fig.show()
fig.write_image("doc/report/images/results/avg_wt_jockey_b.png", scale=IMAGE_SCALE)

In [24]:
# Average waiting time

fig =  go.Figure()

fig = add_simulation_to_plot(fig, df_simulations, 'validazione_1', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'coda1_jockey1', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'coda1_jockey2', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'validazione_3', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'coda4_jockey1', "Avg_waiting_times_standard")

fig = decorate_figure(fig,
    title="Average waiting time - best simulations with jockey",
    y_axis_title="Avg waiting time (min)")

fig.show()
fig.write_image("doc/report/images/results/avg_wt_jockey_best.png", scale=IMAGE_SCALE)

In [25]:
# Average waiting time

fig =  go.Figure()

dash_line=dict(dash='dash')

fig = add_simulation_to_plot(fig, df_simulations, 'coda1_jockey1_5', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'codacondivisa_5', "Avg_waiting_times_standard", line=dash_line)

fig = add_simulation_to_plot(fig, df_simulations, 'coda1_jockey1_6', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'codacondivisa_6', "Avg_waiting_times_standard", line=dash_line)

fig = add_simulation_to_plot(fig, df_simulations, 'coda1_jockey1_7', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'codacondivisa_7', "Avg_waiting_times_standard", line=dash_line)

fig = add_simulation_to_plot(fig, df_simulations, 'coda1_jockey1_8', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'codacondivisa_8', "Avg_waiting_times_standard", line=dash_line)


fig = decorate_figure(fig,
    title="Average waiting time - jockey and parallel queues vs N-fork queue",
    y_axis_title="Avg waiting time (min)", y_dtick=2)

fig.show()
fig.write_image("doc/report/images/results/avg_wt_codacondivisa.png", scale=IMAGE_SCALE)

In [26]:
# Average waiting time

fig =  go.Figure()

fig = add_simulation_to_plot(fig, df_simulations, 'coda1_jockey1', "Avg_waiting_times_standard")
fig = add_simulation_to_plot(fig, df_simulations, 'self_scan', "Avg_waiting_times_self_scan")

fig = decorate_figure(fig,
    title="Average waiting time - standard cashdesks vs self-scan cashdesks",
    y_axis_title="Avg waiting time (min)")

fig.show()
fig.write_image("doc/report/images/results/avg_wt_selfscan.png", scale=IMAGE_SCALE)

In [27]:
# Average waiting time
fig =  go.Figure()

fig = add_simulation_to_plot(
    fig, df_simulations, 'prob_coda1_jockey1', "Avg_waiting_times_standard",
    line=dict(color=COLORS[0], dash='dot'))
fig = add_simulation_to_plot(
    fig, df_simulations, 'prob_coda1_jockey2', "Avg_waiting_times_standard",
    line=dict(color=COLORS[1], dash='dot'))
fig = add_simulation_to_plot(
    fig, df_simulations, 'prob_coda2_jockey1', "Avg_waiting_times_standard",
    line=dict(color=COLORS[2], dash='dot'))
fig = add_simulation_to_plot(
    fig, df_simulations, 'prob_coda2_jockey2', "Avg_waiting_times_standard",
    line=dict(color=COLORS[3], dash='dot'))
fig = add_simulation_to_plot(
    fig, df_simulations, 'coda1_jockey1', "Avg_waiting_times_standard",
    line=dict(color=COLORS[0]))
fig = add_simulation_to_plot(
    fig, df_simulations, 'coda1_jockey2', "Avg_waiting_times_standard",
    line=dict(color=COLORS[1]))
fig = add_simulation_to_plot(
    fig, df_simulations, 'coda2_jockey1', "Avg_waiting_times_standard",
    line=dict(color=COLORS[2]))
fig = add_simulation_to_plot(
    fig, df_simulations, 'coda2_jockey2', "Avg_waiting_times_standard",
    line=dict(color=COLORS[3]))


fig = decorate_figure(fig,
    title="Average waiting time - probabilistic vs deterministic",
    y_axis_title="Avg waiting time (min)")

fig.show()
fig.write_image("doc/report/images/results/avg_wt_prob.png", scale=IMAGE_SCALE)


In [28]:
feature='Density_standard'
x_axis='Avg_waiting_times_standard'

ns = ['coda1_jockey1', 'self_scan']

fig = go.Figure()
fig = add_multiple_traces(fig, df_simulations, ['coda1_jockey1'],
                          'Avg_waiting_times_standard', 'Density_standard', AGGREGATION_PARAMETER)
fig = add_multiple_traces(fig, df_simulations, ['self_scan'],
                          'Avg_waiting_times_self_scan', 'Density_self_scan', AGGREGATION_PARAMETER)
fig = decorate_figure(fig,
    title="KDE (Kernel Density Estimates) - standard cashdesks vs self-scan cashdesks",
    x_axis_title='Avg waiting time (min)', y_axis_title="Density (customer/cashdesk)", x_dtick=1)

fig.show()
fig.write_image("doc/report/images/results/kde_self_scan.png", scale=IMAGE_SCALE)


C:\Users\lucre\AppData\Local\Temp/ipykernel_14016/866377515.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [29]:
feature='Density_standard'
x_axis='Avg_waiting_times_standard'

ns = ['codacondivisa_5', 'codacondivisa_6', 'codacondivisa_7', 'codacondivisa_8',
      'coda1_jockey1_5', 'coda1_jockey1_6', 'coda1_jockey1_7', 'coda1_jockey1_8']

fig = go.Figure()
fig = add_multiple_traces(fig, df_simulations, ns, x_axis, feature, AGGREGATION_PARAMETER)
fig = decorate_figure(fig,
    title="KDE (Kernel Density Estimates) - jockey and parallel queues vs N-fork queue",
    x_axis_title='Avg waiting time (min)', y_axis_title="Density (customer/cashdesk)", x_dtick=5, y_dtick=5)

fig.show()
fig.write_image("doc/report/images/results/kde_codacondivisa.png", scale=IMAGE_SCALE)

C:\Users\lucre\AppData\Local\Temp/ipykernel_14016/866377515.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [30]:
feature='Density_standard'
x_axis='Avg_waiting_times_standard'

ns = ['validazione_1', 'validazione_2', 'coda1_jockey1', 'coda1_jockey2', 'coda2_jockey1', 'coda2_jockey2']

fig = go.Figure()
fig = add_multiple_traces(fig, df_simulations, ns, x_axis, feature, AGGREGATION_PARAMETER)
fig = decorate_figure(fig,
    title="KDE (Kernel Density Estimates) - simulations with jockey A",
    x_axis_title='Avg waiting time (min)', y_axis_title="Density (customer/cashdesk)")

fig.show()
fig.write_image("doc/report/images/results/kde_jockey_a.png", scale=IMAGE_SCALE)

C:\Users\lucre\AppData\Local\Temp/ipykernel_14016/866377515.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

